In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import IPython
import requests
import dotenv
import pandas as pd
import numpy as np

from backend.location import Location, make_grid
from backend import gmaps

dotenv.load_dotenv()

In [ ]:
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
# zurich = Location(47.378, 8.540)
zurich = Location(47.378, 8.540)
# zurich = Location(45, 130)

In [ ]:
zoom = 14
grid = make_grid(zurich, zoom, size=9)

image = get_static_map(zurich, zoom, markers=grid)

IPython.display.Image(image)

In [ ]:
matrix = list(gmaps.get_distance_matrix(grid, grid))

In [ ]:
matrix

In [ ]:
len(matrix)

In [ ]:
with open("9x9matrix-v2.json", "w") as f:
    json.dump(matrix, f)

In [ ]:
with open("9x9matrix-request.json", "w") as f:
    json.dump(gmaps.get_distance_matrix_api_payload(grid, grid), f)

## Analysis

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    # Radius of the Earth in meters
    radius = 6371.0 * 1000  # Earth's mean radius

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = radius * c

    return distance

In [ ]:
with open("../frontend/src/assets/9x9matrix.json") as f:
    matrix = json.load(f)

In [ ]:
df = pd.DataFrame(matrix)
df = df.set_index(["originIndex", "destinationIndex"]).sort_index()

In [ ]:
df["durationSec"] = df["duration"].apply(lambda x: x if pd.isna(x) else int(x[:-1]))

In [ ]:
df["originLat"] = df.index.get_level_values(0).map(lambda i: grid[i].lat)
df["originLng"] = df.index.get_level_values(0).map(lambda i: grid[i].lng)
df["destinationLat"] = df.index.get_level_values(1).map(lambda i: grid[i].lat)
df["destinationLng"] = df.index.get_level_values(1).map(lambda i: grid[i].lng)

In [ ]:
df

In [ ]:
df["sphericalDistance"] = df.apply(lambda row: haversine_distance(row.originLat, row.originLng, row.destinationLat, row.destinationLng), axis=1)

In [ ]:
df["distanceRatio"] =  df["sphericalDistance"] / df["distanceMeters"]
df["timeRatio"] =  df["sphericalDistance"] / df["durationSec"]

In [ ]:
df.sort_values("distanceRatio", ascending=False)

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(df, x="distanceRatio")

In [ ]:
px.histogram(df, x="timeRatio")

In [ ]:
px.scatter(df, "distanceRatio", "timeRatio")

In [ ]:
px.histogram(df, x="distanceRatio")

In [ ]:
df

In [ ]:
df["velocity"] = df["distanceMeters"] / df["duration_sec"]

In [ ]:
df.query("~velocity.isna()").sort_values("velocity", ascending=False)